https://www.kaggle.com/ryanxjhan/cbc-news-coronavirus-articles-march-26
https://github.com/cjhutto/vaderSentiment#about-the-scoring

http://openaccess.uoc.edu/webapps/o2/bitstream/10609/81435/6/jsobrinosTFM0618memoria.pdf

In [1]:
import nltk
from gensim import models
import pandas 
import json
import numpy
import requests
from random import randint, uniform,random

#Análisis de sentimientos
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer



#Traductor
from nltk.translate import AlignedSent, Alignment

import matplotlib.pyplot as plt

##### Editamos el LEXICON al lexicon que viene integrado en SentimentalIntensityAnalizer 

In [2]:
lexicon = pandas.read_csv('SEL/SEL_UTF8.txt', sep='\t', names=['Palabra', 'PFA', 'Categoría'])
lexicon.drop(['Categoría'], axis='columns', inplace=True)
lexicon.info()

Palabras = numpy.array(lexicon.Palabra)
PFA = numpy.array(lexicon.PFA)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2037 entries, 0 to 2036
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Palabra  2037 non-null   object
 1   PFA      2037 non-null   object
dtypes: object(2)
memory usage: 32.0+ KB


In [3]:
lista = []
for i in range(1, len(Palabras)):
    lista.append([Palabras[i], float(PFA[i])])
    
lexi = {a : b for a,b in lista}
lexi

{'abundancia': 0.83,
 'acabalar': 0.396,
 'acallar': 0.198,
 'acatar': 0.198,
 'acción': 0.397,
 'aceptable': 0.594,
 'aceptación': 0.696,
 'acicate': 0.429,
 'aclamación': 0.799,
 'aclamar': 0.799,
 'acogedor': 0.83,
 'acoger': 0.729,
 'acomodadamente': 0.729,
 'acuciar': 0.264,
 'acucioso': 0.232,
 'adecuar': 0.331,
 'adicto': 0.429,
 'admirable': 0.73,
 'admirablemente': 0.663,
 'admiración': 0.764,
 'admirar': 0.73,
 'admitir': 0.53,
 'adorable': 0.898,
 'adorablemente': 0.865,
 'adoración': 0.765,
 'adorador': 0.664,
 'adorar': 0.764,
 'afable': 0.696,
 'afán': 0.764,
 'afectivo': 0.864,
 'afecto': 0.899,
 'afectuosamente': 0.966,
 'afectuosidad': 0.932,
 'afectuoso': 0.898,
 'afervorizar': 0.731,
 'afición': 0.729,
 'aficionar': 0.595,
 'afortunadamente': 0.831,
 'afortunado': 0.932,
 'agradable': 0.899,
 'agradar': 0.798,
 'agradecer': 0.763,
 'agradecido': 0.764,
 'agudeza': 0.331,
 'aguzar': 0.199,
 'ahínco': 0.73,
 'ahíto': 0.564,
 'airoso': 0.565,
 'alabar': 0.53,
 'alboroza

##### Integramos el nuevo Lexicon al lexicon base 

In [5]:
SIA = SentimentIntensityAnalyzer()
SIA.lexicon.update(lexi)

Librería vaderSentiment
* neu=neutral
* compound= es una regla que normaliza, digamos que puede ser parecido a una media de los otros valores
* pos=positivo
* neg=negativo

En general podemos decir:

* positivo sentimiento: compound >= 0.5
* neutral sentimiento: (compound > -0.5) and (compound < 0.5)
* negativo sentimiento: compound <= -0.5

In [110]:
def ClasificadorSentimientos(dataFrame):
    arrAux = numpy.array(dataFrame.Valor)
    indice = numpy.where(arrAux == max(arrAux))[0][0]

    
    if indice == 0:
        print("Negativo")
    elif indice == 1:
        print("Neutro")
    elif indice == 2:
        print("Positivo")

#### Verificamos la precisión del analizador de sentimientos con el nuevo lexicon

In [10]:
tweets = pandas.read_csv("TweetsClasificados.csv", names=['Tweet', 'Polaridad'])
tweets =tweets.dropna()
tweets.info()
tw = numpy.asarray(tweets.Tweet)
pol = numpy.asarray(tweets.Polaridad)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 989 entries, 0 to 1976
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Tweet      989 non-null    object
 1   Polaridad  989 non-null    object
dtypes: object(2)
memory usage: 23.2+ KB


In [40]:
cleanTweets = []
for i in range(len(pol)):
    if (pol[i] != 'NONE'):
        cleanTweets.append(([tw[i], pol[i]]))
        
CT = numpy.asarray(cleanTweets)
print("Tweets totales = ", len(CT))

positivos = 0
negativos = 0
neutrales = 0
for i in range(len(CT)):
    if (CT[i][1] == 'P'):
        positivos += 1
    if (CT[i][1] == 'N'):
        negativos += 1
    if (CT[i][1] == 'NEU'):
        neutrales += 1
print("Positivos = ", positivos, "Negativos = ", negativos, "Neutrales = ", neutrales)

Tweets totales =  896
Positivos =  312 Negativos =  505 Neutrales =  79


In [97]:
Pos = 0
Neg = 0
Neu = 0
for i in range(len(CT)):
    tweet = CT[i][0]
    vs = SIA.polarity_scores(tweet)
    auxArr = numpy.split(numpy.asarray(list(vs.items())).T, 2)
    values = numpy.asarray(auxArr[1], dtype =float).flatten()
    compound = values[3]
    
    if(compound >= 0.05 and CT[i][1] == 'P'):
        Pos += 1
    if(compound <= 0.05 and CT[i][1] == 'N'):
        Neg += 1
    if (compound > -0.05 and compound < 0.05 and CT[i][1] == 'NEU'):
        Neu += 1
        
print("VP(Positivo) = ", Pos, "VP(Negativo) = ", Neg, "VP(Neutral) = ", Neu)#, "Pos1 = ", Pos1)

VP(Positivo) =  132 VP(Negativo) =  360 VP(Neutral) =  20


In [98]:
vs = SIA.polarity_scores(":)")
print(vs)

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4588}
